<a href="https://colab.research.google.com/github/alupo933/Python-Work-for-Bootcamp/blob/main/Pre_Processing_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer

In [38]:
# Read in the data
insurance_df = pd.read_csv('/content/drive/MyDrive/Coding Dojo Data Science Boot Camp/Excel Files/insurance.csv')
insurance_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [39]:
# Define Features (x) & (y)
y = insurance_df['charges']
X = insurance_df.drop(columns='charges')

In [40]:
# Train test split the data to prepare for machine learninig
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [41]:
X.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
dtype: object

## Identify each feature as numerical, ordinal, or nominal. 

age = Numerical

sex = nominal

bmi = numerical

children = numerical

smoker = nominal

region = nominal


## One Hot Encode the nominal features

In [56]:
# make categorical and numeric selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

## Select Categorical Columns ##
cat_data = X_train[cat_selector(X_train)]
cat_data

,sex,smoker,region
693,male,no,northwest
1297,female,no,southeast
634,male,no,southwest
1022,male,yes,southeast
178,female,no,southwest
...,...,...,...
1095,female,no,northeast
1130,female,no,southeast
1294,male,no,northeast
860,female,yes,southwest


In [43]:
from sklearn.preprocessing import OneHotEncoder

# Instantiate OneHotEncoder
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encoder.fit(cat_data)
cat_ohe = ohe_encoder.transform(cat_data)
cat_ohe

array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

## Change Numpy Arrray back to a DF

In [49]:
 pd.DataFrame(cat_ohe, columns=ohe_encoder.get_feature_names(cat_data.columns))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
998,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
999,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1001,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [53]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [54]:
# instantiate the encoder
scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [55]:
# setup Tuples
cat_tuple = (encoder, cat_selector)

In [58]:
col_trans = make_column_transformer(cat_tuple, remainder='passthrough')

## Instantiate a Linear Regression Model

In [50]:
from sklearn.linear_model import LinearRegression

In [51]:
# Make a linear regression instance 
reg = LinearRegression()

In [59]:
# Make a pipleline
pipe = make_pipeline(col_trans, reg)

## Fit the model on the training data

In [60]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f730a101710>)])),
                ('linearregression', LinearRegression())])

##Evaluate the model performance on both the training set and the test set using R^2 score

In [65]:
# Score the training
pipe.score(X_train, y_train)

0.7449555328228536

In [66]:
#Score the testing
pipe.score(X_test, y_test)

0.7672642952734355

In [67]:
from sklearn.metrics import mean_absolute_error

#create predictions form the model on the training data
training_predictions = pipe.predict(X_train)

# Evaluate the model's prediction on the Training Set
# use the MAE metric
training_mae = mean_absolute_error(y_train, training_predictions)
print(f'MAE of the training set is {training_mae: .2f} dollars')

MAE of the training set is  4183.15 dollars
